In [1]:
import torch
import torch.nn as nn
import torch.onnx
import numpy as np
import onnx
import onnxruntime

In [2]:
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.LSTM(13, 20, batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(20, 7)
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x

In [3]:
torch_F = torch.load("word.pt", weights_only=False)
print(torch_F)

NN(
  (rnn): LSTM(13, 20, batch_first=True)
  (f): Sequential(
    (0): Linear(in_features=20, out_features=7, bias=True)
  )
)


In [4]:
x = torch.randn(1,50,13)
dynamic_axes = {"input" : {0 : "b", 1 : "f"}, "output" : {0 : "b", 1 : "f"}}

torch_F.eval()
torch.onnx.export(
    torch_F,
    x,
    "word.onnx",
    export_params=True,
    opset_version=10,
    do_constant_folding=True,
    input_names = ["input"],
    output_names = ["output"],
    dynamic_axes= dynamic_axes
)

C:\ProgramData\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4277: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
onnx_F = onnxruntime.InferenceSession("word.onnx", providers=["CPUExecutionProvider"])

np_x = np.random.randn(2,25,13).astype(np.float32)
tensor_x = torch.tensor(np_x, dtype = torch.float)

tensor_y = torch_F(tensor_x)
inputs = {onnx_F.get_inputs()[0].name : np_x}
np_y = onnx_F.run(None, inputs)

print("troch :", tensor_y.detach().numpy())
print("onnx :", np_y)


troch : [[-0.78478944  0.46747488 -0.11910145 -0.18749915  0.09459582  0.21879351
  -0.36233234]
 [ 0.5598762   0.5391948   0.04662143 -0.73389095 -0.44629106 -0.5101722
   0.21321933]]
onnx : [array([[-0.7847895 ,  0.46747476, -0.11910145, -0.18749917,  0.09459579,
         0.21879342, -0.3623323 ],
       [ 0.5598761 ,  0.5391948 ,  0.04662149, -0.7338909 , -0.44629103,
        -0.5101723 ,  0.2132193 ]], dtype=float32)]
